In [1]:
from collections import defaultdict
from gensim import corpora

documents = [
    "Human machine interface for lab abc computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
]

# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [
    [word for word in document.lower().split() if word not in stoplist]
    for document in documents
]

# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

C:\Users\user\miniconda3\envs\pytorch\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


### num_top=2

In [2]:
from gensim import models
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)

In [3]:
doc = "Human computer interaction"
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]  # convert the query to LSI space
print(vec_lsi)

[(0, 0.4618210045327159), (1, -0.07002766527900021)]


In [4]:
from gensim import similarities
index = similarities.MatrixSimilarity(lsi[corpus])

In [5]:
sims = index[vec_lsi]  # perform a similarity query against the corpus
print(list(enumerate(sims)))  # print (document_number, document_similarity) 2-tuples

[(0, 0.998093), (1, 0.93748635), (2, 0.9984453), (3, 0.9865886), (4, 0.90755945), (5, -0.12416792), (6, -0.10639259), (7, -0.09879464), (8, 0.050041765)]


In [6]:
sims = sorted(enumerate(sims), key=lambda item: -item[1])
for doc_position, doc_score in sims:
    print(doc_score, documents[doc_position])

0.9984453 The EPS user interface management system
0.998093 Human machine interface for lab abc computer applications
0.9865886 System and human system engineering testing of EPS
0.93748635 A survey of user opinion of computer system response time
0.90755945 Relation of user perceived response time to error measurement
0.050041765 Graph minors A survey
-0.09879464 Graph minors IV Widths of trees and well quasi ordering
-0.10639259 The intersection graph of paths in trees
-0.12416792 The generation of random binary unordered trees


### num_top=5

In [7]:
from gensim import models
lsi2 = models.LsiModel(corpus, id2word=dictionary, num_topics=5)

In [8]:
vec_lsi2 = lsi2[vec_bow]  # convert the query to LSI space
print(vec_lsi2)

[(0, 0.46182100453271624), (1, -0.07002766527900016), (2, -0.12452907551899134), (3, 1.0097125584438558), (4, -0.21303040605626491)]


In [9]:
from gensim import similarities
index2 = similarities.MatrixSimilarity(lsi2[corpus])

In [10]:
sims2 = index2[vec_lsi2]  # perform a similarity query against the corpus
print(list(enumerate(sims2)))  # print (document_number, document_similarity) 2-tuples

[(0, 0.9727242), (1, 0.31977737), (2, 0.30448094), (3, 0.24447846), (4, -0.04838742), (5, -0.16497499), (6, -0.09127023), (7, -0.03274583), (8, 0.15309833)]


In [11]:
sims2 = sorted(enumerate(sims2), key=lambda item: -item[1])
for doc_position, doc_score in sims2:
    print(doc_score, documents[doc_position])

0.9727242 Human machine interface for lab abc computer applications
0.31977737 A survey of user opinion of computer system response time
0.30448094 The EPS user interface management system
0.24447846 System and human system engineering testing of EPS
0.15309833 Graph minors A survey
-0.03274583 Graph minors IV Widths of trees and well quasi ordering
-0.04838742 Relation of user perceived response time to error measurement
-0.09127023 The intersection graph of paths in trees
-0.16497499 The generation of random binary unordered trees


### Tf-idf -> LSA

In [12]:
tfidf = models.TfidfModel(corpus) 
print(tfidf[vec_bow])

[(0, 0.7071067811865476), (1, 0.7071067811865476)]


In [13]:
corpus_tfidf = tfidf[corpus]

In [14]:
lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2)  # initialize an LSI transformation
corpus_lsi = lsi_model[corpus_tfidf]

In [15]:
for doc, as_text in zip(corpus_lsi, documents):
    print(doc, as_text)

[(0, 0.06600783396090493), (1, -0.5200703306361845)] Human machine interface for lab abc computer applications
[(0, 0.1966759285914266), (1, -0.7609563167700043)] A survey of user opinion of computer system response time
[(0, 0.08992639972446609), (1, -0.7241860626752505)] The EPS user interface management system
[(0, 0.0758584765217832), (1, -0.6320551586003427)] System and human system engineering testing of EPS
[(0, 0.1015029918498025), (1, -0.5737308483002954)] Relation of user perceived response time to error measurement
[(0, 0.7032108939378309), (1, 0.1611518021402598)] The generation of random binary unordered trees
[(0, 0.877478767311983), (1, 0.16758906864659623)] The intersection graph of paths in trees
[(0, 0.9098624686818577), (1, 0.14086553628719217)] Graph minors IV Widths of trees and well quasi ordering
[(0, 0.6165825350569283), (1, -0.053929075663892635)] Graph minors A survey


In [16]:
index = similarities.MatrixSimilarity(lsi_model[corpus])
vec_tfidf = tfidf[vec_bow]
vec_lsi = lsi_model[vec_tfidf]
sims = index[vec_lsi]
sims = sorted(enumerate(sims), key=lambda item: -item[1])
for doc_position, doc_score in sims:
    print(doc_score, documents[doc_position])

0.9999428 The EPS user interface management system
0.9999408 Human machine interface for lab abc computer applications
0.999879 System and human system engineering testing of EPS
0.99935204 Relation of user perceived response time to error measurement
0.9946708 A survey of user opinion of computer system response time
0.1938726 Graph minors A survey
-0.023664713 Graph minors IV Widths of trees and well quasi ordering
-0.0515742 The intersection graph of paths in trees
-0.08804217 The generation of random binary unordered trees
